# Sparsity metric
How sparse are the relevance maps is measured by using the saliency metric proposed in https://arxiv.org/pdf/2201.13291.pdf:

$$\textrm{r'} = \frac{r -r_{\textrm{min}}}{r_{\textrm{min}} - r_{\textrm{max}}} \quad \textrm{Sparsity} = \frac{1}{r'_{\textrm{mean}}}$$   



In [1]:
from data_constructor import Dataset
from model_constructor import Model
from interpreter_constructor import Interpreter
import numpy as np
import matplotlib.pyplot as plt

train_fraction = 0.8
seed_int = 54321 # 54321

subset = 'test'

DATASET_LABELS = {'helix_1_start':"Sine",'helix_matching_random_amp_feat_time_no_phase':"SineRandom",'helix_matching_random_amp_feat_time':"SineRandomPhase","blink_EEG":"Blink EEG"}
MODELS_LABELS  = {'relu-mlp':'MLP LLM','conv-relu-mlp':'LICOR','CAM-conv':'CNN CAM'} 

AUC_METRICS = []
N_splits = 5
datasets = ['helix_1_start','helix_matching_random_amp_feat_time_no_phase','helix_matching_random_amp_feat_time','blink_EEG'] #'circles','moons','helix_1_start','helix_2_start','helix_1_random_time','helix_2_random_time','helix_random_feat_random_time','helix_matching_random_amp_feat_time', 'helix_matching_random_amp_feat_time_no_phase'
models = ['CAM-conv','conv-relu-mlp','relu-mlp'] # 'relu-mlp', 'conv-relu-mlp', 'CAM-conv'
SPARSITY = []
for i, model_str in enumerate(models):
    for ii, data_str in enumerate(datasets):
        data = Dataset(data_str, train_fraction, seed_int)
        model = Model(seed_int, data, model_str)
        _,test_auc = model.train_model()
        idx_list = range(199) 
        interpreter = Interpreter(data, model, idx_list, subset='test',use_bias=True)
        SPARSITY.append([model_str,data_str,interpreter.getSparsity()])



2023-06-21 07:49:34.704212: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2023-06-21 07:49:35.814901: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2023-06-21 07:49:35.815389: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2023-06-21 07:49:35.849839: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-06-21 07:49:35.850323: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:06:00.0 name: NVIDIA GeForce RTX 2070 SUPER computeCapability: 7.5
coreClock: 1.815GHz coreCount: 40 deviceMemorySize: 7.79GiB deviceMemoryBandwidth: 417.29GiB/s
2023-06-21 07:49:35.850345: I tensorflow/stream_executor/platf

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 72, 1, 5)]        0         
_________________________________________________________________
Conv1 (Conv2D)               (None, 72, 1, 16)         256       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 36, 1, 16)         0         
_________________________________________________________________
Conv2 (Conv2D)               (None, 36, 1, 32)         1568      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 18, 1, 32)         0         
_________________________________________________________________
ConvOutput (Conv2D)          (None, 18, 1, 64)         6208      
_________________________________________________________________
global_average_pooling2d (Gl (None, 64)                0     

2023-06-21 07:49:36.644021: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10
2023-06-21 07:49:36.767983: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.7
2023-06-21 07:49:37.356468: W tensorflow/stream_executor/gpu/asm_compiler.cc:63] Running ptxas --version returned 256
2023-06-21 07:49:37.399388: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] Internal: ptxas exited with non-zero error code 256, output: 
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.


250/250 [==============================] - 6s 9ms/step - loss: 0.5025 - auc: 0.8324 - val_loss: 0.0084 - val_auc: 1.0000
Epoch 2/100
250/250 [==============================] - 1s 2ms/step - loss: 0.0050 - auc: 1.0000 - val_loss: 0.0014 - val_auc: 1.0000
Epoch 3/100
250/250 [==============================] - 1s 2ms/step - loss: 0.0011 - auc: 1.0000 - val_loss: 5.2448e-04 - val_auc: 1.0000
Epoch 4/100
250/250 [==============================] - 1s 2ms/step - loss: 4.2412e-04 - auc: 1.0000 - val_loss: 2.7578e-04 - val_auc: 1.0000
Epoch 5/100
250/250 [==============================] - 1s 2ms/step - loss: 2.2647e-04 - auc: 1.0000 - val_loss: 1.7104e-04 - val_auc: 1.0000
Epoch 6/100
250/250 [==============================] - 1s 2ms/step - loss: 1.3910e-04 - auc: 1.0000 - val_loss: 1.1809e-04 - val_auc: 1.0000
63/63 - 0s - loss: 0.0084 - auc: 1.0000
Test AUC score helix_1_start: 1.0
Model: "model_2"
_________________________________________________________________
Layer (type)                 

In [ ]:
import pandas as pd
df_sparsity = pd.DataFrame(SPARSITY, columns = ["Model", "Dataset","Sparsity"])
df_sparsity.pivot(index='Model',columns='Dataset')

Sparsity                                                    \
Dataset       blink_EEG helix_1_start helix_matching_random_amp_feat_time   
Model                                                                       
CAM-conv       0.015074      0.025126                            0.025126   
conv-relu-mlp  0.002281      0.016425                            0.020674   
relu-mlp       0.010944      0.021496                            0.023411   

                                                            
Dataset       helix_matching_random_amp_feat_time_no_phase  
Model                                                       
CAM-conv                                          0.025126  
conv-relu-mlp                                     0.008119  
relu-mlp                                          0.023972